In [ ]:
import numpy as np 
import pandas as pd 
from PIL import Image
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.utils import class_weight
from sklearn.preprocessing import minmax_scale
import random
import cv2
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers.experimental.preprocessing import RandomCrop,CenterCrop, RandomRotation
from tensorflow.keras.optimizers import Adam

In [ ]:
training_folder = '../input/cassava-leaf-disease-classification/train_images/' #훈련할 이미지들이 있는 폴더
samples_df = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv') #훈련할 이미지의 이름 및 각 label 데이터 로드
samples_df["filepath"] = training_folder+samples_df["image_id"] #사진을 불러오기 쉽도록 폴더와 이미지의 이름을 합쳐 경로를 생성
samples_df = samples_df.drop(['image_id'],axis=1) #필요없는 이미지 이름을 모두 버림

In [ ]:
#이미지 파일을 가져오기 쉽게 파일이름+이미지이름을 합해서 저장
samples_df['filepath'][0]

In [ ]:
samples_df = shuffle(samples_df, random_state=42) #데이터를 무작위로 섞음
train_size = int(len(samples_df)*0.8) # 훈련에 사용할 데이터의 크기를 지정
training_df = samples_df[:train_size] # 훈련 데이터셋을 만들어줌
validation_df = samples_df[train_size:] # validation 데이터셋을 만들어줌

In [ ]:
batch_size = 8 # 배치 사이즈를 설정
image_size = 512 # 이미지의 크기를 설정
input_shape = (image_size, image_size, 3) #이미지의 사이즈 정의 (컬러 이미지이기 때문에 한 화소당 3개의 데이터가 필요)
dropout_rate = 0.4 #드롭아웃 비율 정의
classes_to_predict = sorted(training_df.label.unique()) #예측해야 하는 클래스 수 정의, 여기서는 5개

In [ ]:
"""
train 데이터와 validation 데이터를 텐서플로우 Dataset으로 정의합니다.
텐서플로우 Dataset는 동적으로 데이터를 불러와, 너무 많은 데이터가 메모리에 쓰여지는 일을 방지하여 퍼포먼스가 향상됩니다.
더 자세한 내용은 아래의 링크를 참조하세요.
https://www.tensorflow.org/guide/data_performance?hl=ko
"""
training_data = tf.data.Dataset.from_tensor_slices((training_df.filepath.values, training_df.label.values))
validation_data = tf.data.Dataset.from_tensor_slices((validation_df.filepath.values, validation_df.label.values))
#텐서플로우 Dataset
# 하나의 학습단계를 실행할 때 필요한 시간을 급격히 줄이기 위해 GPU나 TPU를 사용.
#최대 성능을 위해서는 현재 단계가 종료되기 전에 다음 스텝의 데이터를 운반하는 효율적인 입력 파이프라인이 필요
# tf.data api가 유연하고 효율적인 입력 파이프라인을 만드는데 도움

In [ ]:
def load_image_and_label_from_path(image_path, label): #이미지 데이터를 불러와 텐서 (array와 비슷한 형태)로 변환하는 함수
    img = tf.io.read_file(image_path) #이미지 경로의 파일을 읽음
    img = tf.image.decode_jpeg(img, channels=3) #이미지를 array 데이터로 변환하여 저장
    img = tf.image.random_crop(img, size=[image_size,image_size,3]) # 이미지를 랜덤으로 원하는 사이즈로 잘라줌. 중앙만 자르고 싶다면 central_crop 사용.
    return img, label

AUTOTUNE = tf.data.experimental.AUTOTUNE #메모리 동적 할당을 위한 AUTOTUNE
training_data = training_data.map(load_image_and_label_from_path, num_parallel_calls=AUTOTUNE) #train 데이터를 불러옴
validation_data = validation_data.map(load_image_and_label_from_path,num_parallel_calls=AUTOTUNE) #validation 데이터를 불러옴

In [ ]:
#train 및 validation 데이터를 훈련하기 좋게 batch로 자름
training_data_batches = training_data.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=AUTOTUNE)
validation_data_batches = validation_data.shuffle(buffer_size=1000).batch(batch_size).prefetch(buffer_size=AUTOTUNE)

In [ ]:
"""
이미지를 Augumentation 해주는 레이어를 만들어줍니다. 모델을 만들 때 augmentation layer을 넣으면 자동으로 이미지를 다양하게 변환하여 줍니다.
더 많은 augumentation을 적용해보고 싶으면 https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing 이 링크를 참조하세요.
또한, imgaug, albumentation과 같은 강력한 augumentation 라이브러리도 살펴보세요. 
"""
data_augmentation_layers = tf.keras.Sequential(
    [
        layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"), #랜덤으로 이미지를 좌우로 뒤집어줌.
        layers.experimental.preprocessing.RandomRotation(0.25), #이미지를 좌우로 25% 이내로 랜덤으로 돌립니다. 
        layers.experimental.preprocessing.RandomZoom((-0.2, 0)), #이미지를 0~20%만큼 랜덤으로 축소합니다.
        
    ]
)


# 모델만들기 및 학습

In [ ]:
"""
이 베이스라인에서는 transfer learning을 사용합니다. 미리 훈련되어 있는 이미지용 모델을 불러와서 그 모델의 뒤쪽에 나만의 모델을 추가한 뒤 학습하는 방식입니다.
직접 수많은 레이어의 모델을 디자인하는 것은 어렵기 때문에 이러한 방법을 사용합니다.
여기서는 구글의 EfficientNetB0를 사용합니다. 이 모델에 대한 자세한 내용은 https://arxiv.org/pdf/1905.11946.pdf 이 논문을 참고하세요.

주의!! imagenet 가중치 값을 다운받기 위하여 우측 상단 |< 표시를 누르고 setting에서 Internet을 켜줘야합니다.
"""
efficientnet = EfficientNetB0(weights="imagenet", #이미지넷 가중치 값을 불러와 적용
                              include_top=False, 
                              input_shape=input_shape, 
                              drop_connect_rate=dropout_rate) #efficientnetB0 모델을 로드
efficientnet.trainable=True # efficientnetb0의 학습을 허용

In [ ]:
"""
자신만의 CNN 모델을 직접 만들어 보아도 괜찮습니다.
"""
model = Sequential() #새 Sequential 모델을 만듬 
model.add(Input(shape=input_shape)) #인풋을 이미지 사이즈로 설정
model.add(data_augmentation_layers) #이미지 augumentation 레이어 추가
model.add(efficientnet) # efficientnetb0 추가
model.add(layers.GlobalAveragePooling2D()) # 풀링 레이어를 추가
model.add(layers.Dropout(dropout_rate)) # 드롭아웃 레이어를 추가
model.add(Dense(len(classes_to_predict), activation="softmax")) #마지막 덴스 레이어를 추가. 예측할 클래스의 개수만큼이 아웃풋이 된다. 
model.summary() #모델 확인

In [ ]:
epochs = 5 #에폭 수를 설정합니다.
decay_steps = int(round(len(training_df)/batch_size))*epochs
cosine_decay = CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.3) #learning rate를 에폭이 지날수록 점점 줄여나가는 cosine decay 방법을 사용합니다. 
callbacks = [ModelCheckpoint(filepath='mymodel.h5', monitor='val_loss', save_best_only=True), #가장 validation loss가 낮은 에폭의 모델을 .h5 파일로 저장합니다. 
            EarlyStopping(monitor='val_loss', patience = 5, verbose=1)] #정해진 에폭이 되기 전에 5번의 에폭동한 validation loss가 향상되지 않으면 학습을 종료합니다. 

model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(cosine_decay), metrics=["accuracy"]) #loss는 sparse_categorical_crossentropy, optimizer는 Adam을 사용합니다. 각 에폭당 정확도를 통해 모델의 성능을 모니터링합니다, 

In [ ]:
history = model.fit(training_data_batches, #모델을 학습합니다. 
                  epochs = epochs, 
                  validation_data=validation_data_batches,
                  callbacks=callbacks)